random forests will only handle tabular data while neural networks cna handle all sorts of data types. A random forest model with a sufficient number of estimators and tree depth should be able to perform at a similar capacity to most deep learning models

In [1]:
#imports 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np  
import tensorflow as tf
from path import Path

In [2]:
#load data
file_path = Path("../Resources/loan_status.csv")
loans_df = pd.read_csv(file_path)
loans_df.head()

,Loan_Status,Current_Loan_Amount,Term,Credit_Score,Annual_Income,Years_in_current_job,Home_Ownership,Purpose,Monthly_Debt,Years_of_Credit_History,Months_since_last_delinquent,Number_of_Open_Accounts,Number_of_Credit_Problems,Current_Credit_Balance,Maximum_Open_Credit,Bankruptcies,Tax_Liens
0,Fully_Paid,99999999,Short_Term,741.0,2231892.0,8_years,Own_Home,Debt_Consolidation,29200.53,14.9,29.0,18,1,297996,750090.0,0.0,0.0
1,Fully_Paid,217646,Short_Term,730.0,1184194.0,<_1_year,Home_Mortgage,Debt_Consolidation,10855.08,19.6,10.0,13,1,122170,272052.0,1.0,0.0
2,Fully_Paid,548746,Short_Term,678.0,2559110.0,2_years,Rent,Debt_Consolidation,18660.28,22.6,33.0,4,0,437171,555038.0,0.0,0.0
3,Fully_Paid,99999999,Short_Term,728.0,714628.0,3_years,Rent,Debt_Consolidation,11851.06,16.0,76.0,16,0,203965,289784.0,0.0,0.0
4,Fully_Paid,99999999,Short_Term,740.0,776188.0,<_1_year,Own_Home,Debt_Consolidation,11578.22,8.5,25.0,6,0,134083,220220.0,0.0,0.0


since random forests and sequential deep learning models both require preprocessing we don't need to keep track of separate scaled and unscaled data

In [3]:
#generate categorical variable list 
loan_cat = loans_df.dtypes[loans_df.dtypes=="object"].index.tolist()
#check number of unique values
loans_df[loan_cat].nunique()

Loan_Status              2
Term                     2
Years_in_current_job    11
Home_Ownership           4
Purpose                  7
dtype: int64

In [4]:
#years in current job has 11 unique values. we should check the number of data points for each unique value to find out if any categorical variables need to be bucketed  

loans_df.Years_in_current_job.value_counts()

10+_years    13149
2_years       3225
3_years       2997
&lt;_1_year      2699
5_years       2487
4_years       2286
1_year        2247
6_years       2109
7_years       2082
8_years       1675
9_years       1467
Name: Years_in_current_job, dtype: int64

In [5]:
#each category has a significant number of values. we don't want to bucket these and potentially confuse the model

#create onehotencoder instance
enc = OneHotEncoder(sparse=False)

#fit and transform using categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(loans_df[loan_cat]))

#add encoded variables to the dataframe
encode_df.columns = enc.get_feature_names(loan_cat)
encode_df.head()

,Loan_Status_Fully_Paid,Loan_Status_Not_Paid,Term_Long_Term,Term_Short_Term,Years_in_current_job_10+_years,Years_in_current_job_1_year,Years_in_current_job_2_years,Years_in_current_job_3_years,Years_in_current_job_4_years,Years_in_current_job_5_years,...,Home_Ownership_Home_Mortgage,Home_Ownership_Own_Home,Home_Ownership_Rent,Purpose_Business_Loan,Purpose_Buy_House,Purpose_Buy_a_Car,Purpose_Debt_Consolidation,Purpose_Home_Improvements,Purpose_Medical_Bills,Purpose_Other
0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [6]:
#merge encoded features and drop originals
loans_df = loans_df.merge(encode_df, left_index=True, right_index=True)
loans_df = loans_df.drop(loan_cat,1)
loans_df.head()

,Current_Loan_Amount,Credit_Score,Annual_Income,Monthly_Debt,Years_of_Credit_History,Months_since_last_delinquent,Number_of_Open_Accounts,Number_of_Credit_Problems,Current_Credit_Balance,Maximum_Open_Credit,...,Home_Ownership_Home_Mortgage,Home_Ownership_Own_Home,Home_Ownership_Rent,Purpose_Business_Loan,Purpose_Buy_House,Purpose_Buy_a_Car,Purpose_Debt_Consolidation,Purpose_Home_Improvements,Purpose_Medical_Bills,Purpose_Other
0,99999999,741.0,2231892.0,29200.53,14.9,29.0,18,1,297996,750090.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,217646,730.0,1184194.0,10855.08,19.6,10.0,13,1,122170,272052.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,548746,678.0,2559110.0,18660.28,22.6,33.0,4,0,437171,555038.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,99999999,728.0,714628.0,11851.06,16.0,76.0,16,0,203965,289784.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,99999999,740.0,776188.0,11578.22,8.5,25.0,6,0,134083,220220.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [8]:
#remove loan status target from feature data
y = loans_df.Loan_Status_Fully_Paid.values
X = loans_df.drop(columns=["Loan_Status_Fully_Paid", "Loan_Status_Not_Paid"]).values

#split into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, stratify=y)

#create standardscaler instance
scaler = StandardScaler()

#fit the scaler
X_scaler = scaler.fit(X_train)

#scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
#create random forest model. 128 estimators is the largest number of estimators we want to use in a model 
rf_model = RandomForestClassifier(n_estimators=128, random_state=78)

#fit the model 
rf_model.fit(X_train_scaled, y_train)

#evaluate the model
y_pred = rf_model.predict(X_test_scaled)
print(f"Random forest predictive accuracy: {accuracy_score(y_test, y_pred):.3f}")

Random forest predictive accuracy: 0.849


In [10]:
#define deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 24
hidden_nodes_layer2 = 12

nn = tf.keras.models.Sequential()

#add first hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

#add second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

#add output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

#compile the model and customize metrics 
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [11]:
#train the deep nn
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

#evaluate the model 
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Train on 27317 samples
Epoch 1/50
27317/27317 [==============================] - 5s 194us/sample - loss: 0.4197 - accuracy: 0.8364
Epoch 2/50
27317/27317 [==============================] - 4s 137us/sample - loss: 0.3860 - accuracy: 0.8490
Epoch 3/50
27317/27317 [==============================] - 3s 107us/sample - loss: 0.3822 - accuracy: 0.8494
Epoch 4/50
27317/27317 [==============================] - 3s 124us/sample - loss: 0.3800 - accuracy: 0.8491
Epoch 5/50
27317/27317 [==============================] - 3s 122us/sample - loss: 0.3786 - accuracy: 0.8494
Epoch 6/50
27317/27317 [==============================] - 3s 113us/sample - loss: 0.3772 - accuracy: 0.8494
Epoch 7/50
27317/27317 [==============================] - 4s 141us/sample - loss: 0.3766 - accuracy: 0.8495
Epoch 8/50
27317/27317 [==============================] - 3s 127us/sample - loss: 0.3761 - accuracy: 0.8495
Epoch 9/50
27317/27317 [==============================] - 3s 111us/sample - loss: 0.3753 - accuracy: 0.8493
Epoch

comparing both models accuracy scores we see that their output is very similar. both were able to predict whether or not a loan will be repaid over 80% of the time. Although their performance was similar, the deep learning model took minutes to train while the random forest took seconds. The random forest was able to achieve comparable results with less code and faster performance. If a dataset is tabular a random forest is a great place to start. 